In [1]:
import torch

In [2]:
inputs=torch.tensor(
    [
     [0.43,0.15,0.89],#your
     [0.55,0.87,0.66],#journey
     [0.57,0.85,0.64],#starts
     [0.22,0.58,0.33],#with
     [0.77,0.25,0.10],#one
     [0.05,0.80,0.55],#step
    ]
)

In [3]:
w_q=torch.nn.Linear(inputs.shape[1],2,bias=False)
print(w_q)
w_k=torch.nn.Linear(inputs.shape[1],2,bias=False)
w_v=torch.nn.Linear(inputs.shape[1],2,bias=False)

Linear(in_features=3, out_features=2, bias=False)


In [4]:
queries=w_q(inputs)
keys=w_k(inputs)
values=w_v(inputs)

print(queries)

tensor([[-0.0929,  0.3304],
        [-0.1690,  0.6231],
        [-0.1644,  0.6046],
        [-0.1047,  0.3915],
        [-0.0342,  0.1024],
        [-0.1554,  0.5903]], grad_fn=<MmBackward0>)


In [5]:
attn_score=queries@keys.T
print(attn_score)

tensor([[-0.1286, -0.0743, -0.0696, -0.0397,  0.0348, -0.0882],
        [-0.2442, -0.1441, -0.1352, -0.0771,  0.0634, -0.1692],
        [-0.2369, -0.1397, -0.1310, -0.0747,  0.0617, -0.1640],
        [-0.1538, -0.0916, -0.0859, -0.0490,  0.0393, -0.1070],
        [-0.0384, -0.0195, -0.0181, -0.0102,  0.0127, -0.0248],
        [-0.2326, -0.1397, -0.1312, -0.0749,  0.0584, -0.1625]],
       grad_fn=<MmBackward0>)


In [6]:
attn_weight=torch.softmax(attn_score/keys.shape[1]**0.5,dim=1)
print(attn_weight)

tensor([[0.1588, 0.1650, 0.1655, 0.1691, 0.1782, 0.1634],
        [0.1521, 0.1632, 0.1642, 0.1711, 0.1890, 0.1603],
        [0.1525, 0.1633, 0.1643, 0.1710, 0.1883, 0.1605],
        [0.1575, 0.1645, 0.1652, 0.1696, 0.1805, 0.1628],
        [0.1641, 0.1663, 0.1665, 0.1674, 0.1701, 0.1657],
        [0.1529, 0.1633, 0.1643, 0.1710, 0.1879, 0.1607]],
       grad_fn=<SoftmaxBackward0>)


In [7]:
#use pytorch tril function to diagonalize
context_length=inputs.shape[0]
mask1=torch.tril(torch.ones(context_length,context_length))
print(mask1)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [8]:
#use element wise multiplication not matrix
masked_attn_score=attn_weight*mask1
print(masked_attn_score)

#but this attention score is not fully masked or optimal becaus the softmax normalization adds influence on its from the previous attn score so use upper negative infinity mask instead of zero

tensor([[0.1588, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1521, 0.1632, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1525, 0.1633, 0.1643, 0.0000, 0.0000, 0.0000],
        [0.1575, 0.1645, 0.1652, 0.1696, 0.0000, 0.0000],
        [0.1641, 0.1663, 0.1665, 0.1674, 0.1701, 0.0000],
        [0.1529, 0.1633, 0.1643, 0.1710, 0.1879, 0.1607]],
       grad_fn=<MulBackward0>)


In [9]:
#normalize the masked attention score  to sum up 1 in each row to get masked attn weight
row_sum=masked_attn_score.sum(dim=1,keepdim=True)
print(row_sum)
masked_attn_weight=masked_attn_score/row_sum
print(masked_attn_weight)

tensor([[0.1588],
        [0.3153],
        [0.4801],
        [0.6568],
        [0.8343],
        [1.0000]], grad_fn=<SumBackward1>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4823, 0.5177, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3176, 0.3402, 0.3423, 0.0000, 0.0000, 0.0000],
        [0.2397, 0.2505, 0.2515, 0.2582, 0.0000, 0.0000],
        [0.1967, 0.1993, 0.1995, 0.2006, 0.2039, 0.0000],
        [0.1529, 0.1633, 0.1643, 0.1710, 0.1879, 0.1607]],
       grad_fn=<DivBackward0>)


In [10]:
masked_context_vectore=masked_attn_weight@values
print(masked_context_vectore)

tensor([[-0.1567,  0.4693],
        [-0.2068,  0.5076],
        [-0.2236,  0.5170],
        [-0.1982,  0.4582],
        [-0.2049,  0.4137],
        [-0.1904,  0.4051]], grad_fn=<MmBackward0>)


# Negative infinity masking to avoid any influence of future attention

In [12]:
print(attn_score)

tensor([[-0.1286, -0.0743, -0.0696, -0.0397,  0.0348, -0.0882],
        [-0.2442, -0.1441, -0.1352, -0.0771,  0.0634, -0.1692],
        [-0.2369, -0.1397, -0.1310, -0.0747,  0.0617, -0.1640],
        [-0.1538, -0.0916, -0.0859, -0.0490,  0.0393, -0.1070],
        [-0.0384, -0.0195, -0.0181, -0.0102,  0.0127, -0.0248],
        [-0.2326, -0.1397, -0.1312, -0.0749,  0.0584, -0.1625]],
       grad_fn=<MmBackward0>)


In [14]:
mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
print(mask)
masked=attn_score.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[-0.1286,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.2442, -0.1441,    -inf,    -inf,    -inf,    -inf],
        [-0.2369, -0.1397, -0.1310,    -inf,    -inf,    -inf],
        [-0.1538, -0.0916, -0.0859, -0.0490,    -inf,    -inf],
        [-0.0384, -0.0195, -0.0181, -0.0102,  0.0127,    -inf],
        [-0.2326, -0.1397, -0.1312, -0.0749,  0.0584, -0.1625]],
       grad_fn=<MaskedFillBackward0>)


In [15]:
masked_attn_weight=torch.softmax(masked/keys.shape[1],dim=1)
print(masked_attn_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4875, 0.5125, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3221, 0.3382, 0.3397, 0.0000, 0.0000, 0.0000],
        [0.2427, 0.2504, 0.2511, 0.2558, 0.0000, 0.0000],
        [0.1976, 0.1995, 0.1997, 0.2004, 0.2028, 0.0000],
        [0.1569, 0.1644, 0.1651, 0.1698, 0.1815, 0.1625]],
       grad_fn=<SoftmaxBackward0>)


# Masking additional attention weight with Dropout

In [16]:
torch.manual_seed(123)
dropout=torch.nn.Dropout(0.5)
print(dropout)
ones=torch.ones(6,6)
print(ones)
print(dropout(ones))

Dropout(p=0.5, inplace=False)
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [17]:
attn_dropout=dropout(attn_weight)
print(attn_dropout)

tensor([[0.3176, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3285, 0.0000, 0.3780, 0.3207],
        [0.0000, 0.3267, 0.3287, 0.3420, 0.3766, 0.3211],
        [0.0000, 0.0000, 0.3304, 0.3391, 0.0000, 0.3255],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3402, 0.3313],
        [0.3058, 0.3266, 0.0000, 0.3419, 0.3757, 0.3214]],
       grad_fn=<MulBackward0>)


# Causal attention Class with Batches

In [18]:
batch=torch.stack((inputs,inputs),dim=0)
print(batch)
print(batch.shape)

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
torch.Size([2, 6, 3])


In [19]:
print(batch.transpose(1,2))#change row(1) with colmn(2)

tensor([[[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]],

        [[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]]])


In [27]:
class Causal_Attention(torch.nn.Module):
    def __init__(self,in_d,out_d,context_length,dropout_percent):
        super().__init__()
        self.w_q=torch.nn.Linear(in_d,out_d)
        self.w_k=torch.nn.Linear(in_d,out_d)
        self.w_v=torch.nn.Linear(in_d,out_d)
        self.dropout=torch.nn.Dropout(dropout_percent)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    
    def forward(self,input_embedding):
        
        b,num_token,d_in=input_embedding.shape
        queries=self.w_q(input_embedding)
        keys=self.w_k(input_embedding)
        values=self.w_v(input_embedding)

        print(keys.shape[-1])

        attn_score=queries@keys.transpose(1,2)

        attn_score.masked_fill_(self.mask.bool()[:num_token, :num_token],-torch.inf)
        
        print('attn_score',attn_score)
        attn_weight=torch.softmax(attn_score/3**0.5,dim=-1)
        print('attn_weight',attn_weight)
        attn_weight=self.dropout(attn_weight)

        masked_context_vectore=attn_weight@values
        return masked_context_vectore

        
        

        
        


In [22]:
in_d,out_d=3,2
print(in_d)

3


In [28]:
torch.manual_seed(123)
context_length=batch.shape[1]
print(context_length)
ca=Causal_Attention(in_d,out_d,context_length,0.0)
print(ca)

ca.forward(batch)


6
Causal_Attention(
  (w_q): Linear(in_features=3, out_features=2, bias=True)
  (w_k): Linear(in_features=3, out_features=2, bias=True)
  (w_v): Linear(in_features=3, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)
2
attn_score tensor([[[0.4271,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.5730, 0.5534,   -inf,   -inf,   -inf,   -inf],
         [0.5703, 0.5495, 0.5453,   -inf,   -inf,   -inf],
         [0.5143, 0.5440, 0.5368, 0.3764,   -inf,   -inf],
         [0.4756, 0.4463, 0.4436, 0.3051, 0.3194,   -inf],
         [0.5452, 0.5841, 0.5760, 0.4047, 0.3213, 0.4979]],

        [[0.4271,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.5730, 0.5534,   -inf,   -inf,   -inf,   -inf],
         [0.5703, 0.5495, 0.5453,   -inf,   -inf,   -inf],
         [0.5143, 0.5440, 0.5368, 0.3764,   -inf,   -inf],
         [0.4756, 0.4463, 0.4436, 0.3051, 0.3194,   -inf],
         [0.5452, 0.5841, 0.5760, 0.4047, 0.3213, 0.4979]]],
       grad_fn=<MaskedFillBackward0>

tensor([[[-0.0960,  0.7940],
         [ 0.0286,  0.9389],
         [ 0.0659,  0.9852],
         [ 0.1066,  0.9598],
         [ 0.0661,  0.9304],
         [ 0.1183,  0.9366]],

        [[-0.0960,  0.7940],
         [ 0.0286,  0.9389],
         [ 0.0659,  0.9852],
         [ 0.1066,  0.9598],
         [ 0.0661,  0.9304],
         [ 0.1183,  0.9366]]], grad_fn=<UnsafeViewBackward0>)

# Multi head attention wrapper class

In [43]:
class MultiHeadAttentionWrapper(torch.nn.Module):

    def __init__(self,in_d,out_d,context_length,dropout_percent,num_head):

        super().__init__()
        self.heads=torch.nn.ModuleList([Causal_Attention(in_d,out_d,context_length,dropout_percent) for _ in range(num_head)])

    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads],dim=-1)

In [44]:
torch.manual_seed(123)
context_length=batch.shape[1]
print(context_length)
mha=MultiHeadAttentionWrapper(in_d,out_d,context_length,0.0,num_head=2)

context_vec=mha(batch)
print('context_veccccccccc',context_vec)




6
2
attn_score tensor([[[0.4271,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.5730, 0.5534,   -inf,   -inf,   -inf,   -inf],
         [0.5703, 0.5495, 0.5453,   -inf,   -inf,   -inf],
         [0.5143, 0.5440, 0.5368, 0.3764,   -inf,   -inf],
         [0.4756, 0.4463, 0.4436, 0.3051, 0.3194,   -inf],
         [0.5452, 0.5841, 0.5760, 0.4047, 0.3213, 0.4979]],

        [[0.4271,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.5730, 0.5534,   -inf,   -inf,   -inf,   -inf],
         [0.5703, 0.5495, 0.5453,   -inf,   -inf,   -inf],
         [0.5143, 0.5440, 0.5368, 0.3764,   -inf,   -inf],
         [0.4756, 0.4463, 0.4436, 0.3051, 0.3194,   -inf],
         [0.5452, 0.5841, 0.5760, 0.4047, 0.3213, 0.4979]]],
       grad_fn=<MaskedFillBackward0>)
attn_weight tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5028, 0.4972, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3363, 0.3323, 0.3315, 0.0000, 0.0000, 0.0000],
         [0.2529, 0.2573, 0.2562, 0.2336, 0.00